In [1]:
import os

is_kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [2]:
from pathlib import Path

path = Path('us-patent-phrase-to-phrase-matching')

In [3]:
import pandas as pd

df = pd.read_csv(path/'train.csv')
df_eval = pd.read_csv(path/'test.csv')

In [4]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2 ' + df.target + '; ANC1: ' + df.anchor
df_eval['input'] = 'TEXT1: ' + df_eval.context + '; TEXT2 ' + df_eval.target + '; ANC1: ' + df_eval.anchor

In [5]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(df)
ds_eval = Dataset.from_pandas(df_eval) 

In [6]:
model_name = 'microsoft/deberta-v3-small'

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained( model_name )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/sam/miniconda3/envs/fastbook/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
def token_func(x): return tokenizer(x['input'])

token_ds = ds.map(token_func, batched=True)
token_ds = token_ds.rename_column( 'score', 'labels' )

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [16]:
dds = token_ds.train_test_split(0.25, seed=42)

In [10]:
import numpy as np
import torch

def corr(x,y): return torch.sigmoid( torch.tensor( np.corrcoef(x, y)[0][1] ) ).item()

def corr_d(evalpoint): return {'pearson': corr(*evalpoint)}

In [17]:
from transformers import TrainingArguments, Trainer

bs = 128
epochs = 4

lr = 8e-5

args = TrainingArguments( 'outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
                       evaluation_strategy='epoch', per_device_eval_batch_size=bs*2, per_device_train_batch_size=bs,
                       num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                 tokenizer=tokenizer, compute_metrics=corr_d)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [19]:
trainer.train()

/home/sam/miniconda3/envs/fastbook/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.025090,0.689697
2,No log,0.024533,0.695728
3,0.032500,0.022633,0.696778
4,0.032500,0.022931,0.697062


TrainOutput(global_step=856, training_loss=0.02458987559113547, metrics={'train_runtime': 127.241, 'train_samples_per_second': 859.912, 'train_steps_per_second': 6.727, 'total_flos': 688296287637120.0, 'train_loss': 0.02458987559113547, 'epoch': 4.0})

AttributeError: 'DataFrame' object has no attribute 'object'